In [ ]:
# 📦 Imports
!pip install -q ortools
from ortools.sat.python import cp_model
from collections import Counter, defaultdict
import pandas as pd
import itertools
import datetime
from datetime import date, timedelta
import math  # für ceil/floor bei Prozentzielen

DEBUG = True

# ---- Konfig ----
initial_weeks = 13
max_attempts = 5

start_date = date(2025, 10, 1)  # Start des Schichtplans

# Ferien BW (2025/26) – händisch gepflegt
ferien_bw = [
    # Herbstferien 2025
    (date(2025, 10, 27), date(2025, 10, 31)),
    # Weihnachtsferien 2025/2026
    (date(2025, 12, 22), date(2026, 1, 5)),
    # Oster-/Frühjahrsferien 2026
    (date(2026, 3, 30), date(2026, 4, 11)),
    # Himmelfahrt/Pfingsten 2026
    (date(2026, 5, 26), date(2026, 6, 5)),
    # Sommerferien 2026
    (date(2026, 7, 30), date(2026, 9, 12)),
]

days_per_week = ['Mi', 'Do', 'Fr', 'Sa', 'So']
time_slots    = ['14', '17', '20']

# 🔁 Planwoche→Kalenderdatum Mapping (NEU)
DAYS_PER_PLANWEEK = len(days_per_week)
def date_for_d(d: int) -> date:
    # d = 0..num_days-1  → w=Planwoche, k=0..4 (Mi..So)
    w, k = divmod(d, DAYS_PER_PLANWEEK)
    return start_date + timedelta(days=w*7 + k)

employees = [
    "Alisa","Eileen","Eren","Kimi","Kübra","Lea","Lena","Levi","Lucy",
    "Maik","Marianna","Mika","Niko","Semih","Serkan","Wolfi","Xenia"
]
mitarbeiter_index = {name: i for i, name in enumerate(employees)}
num_employees = len(employees)

# Bevorzugte Paare (Namen)
bevorzugte_paare = [
    ("Levi","Lea"),
    ("Alisa","Niko"),
    ("Lena","Marianna"),
]
# Auf Indizes abbilden
bevorzugte_paare_idx = []
seen_pairs = set()
for a_name, b_name in bevorzugte_paare:
    if a_name in mitarbeiter_index and b_name in mitarbeiter_index and a_name != b_name:
        a = mitarbeiter_index[a_name]
        b = mitarbeiter_index[b_name]
        if a > b:
            a, b = b, a   # Normalisieren: kleinerer Index zuerst
        if (a, b) not in seen_pairs:
            bevorzugte_paare_idx.append((a, b))
            seen_pairs.add((a, b))

# Wünsche (soft)
wuensche = {
    0:  [('Sa','14'),('Sa','17'),('Sa','20'),('So','14'),('So','17'),('So','20')],
    6:  [('So','14'),('So','17')],
    8:  [('Fr','20'),('So','14'),('So','17')],
    9:  [('Mi','14'),('Mi','17'),('Mi','20'),('So','14'),('So','17'),('So','20')],
    10: [('Mi','20'),('Do','20'),('So','20')],
    13: [('Sa','20'),('So','14'),('So','17'),('So','20')],
    16: [('Sa','14'),('Sa','17'),('Sa','20')]
}

# Pflichtbesetzung (Normal)
pflichtbesetzung_normal = {
    'Mi': {'14':0,'17':0,'20':2},
    'Do': {'14':0,'17':0,'20':2},
    'Fr': {'14':0,'17':1,'20':2},
    'Sa': {'14':2,'17':2,'20':2},
    'So': {'14':2,'17':2,'20':1},
}
# Pflichtbesetzung (Ferien)
pflichtbesetzung_ferien = {
    'Mi': {'14':0,'17':0,'20':2},
    'Do': {'14':0,'17':0,'20':1},  # abweichend
    'Fr': {'14':0,'17':1,'20':2},
    'Sa': {'14':2,'17':2,'20':2},
    'So': {'14':2,'17':2,'20':1},
}

frequency_rules = [
    {
        "employees": ["Lea"],           # Lea
        "tags": ["Sa","So"],            # Wochenenden
        "times": None,                  # egal welche Uhrzeit
        "window_weeks": 4,              # in jedem 4-Wochen-Block
        "min_count": 0,                 # mindestens 0
        "max_count": 1,                 # höchstens 1 Wochenende
        "count_per_week": "any",        # 0/1 pro Woche
        "sliding": False
    },
]

# Gewichte
W_WISH    = 7
W_SPREAD  = 8
W_SLOTDEV = 5
W_PAIR    = 120
W_PROP_FAIR = 12

# 🔒 Prozentvorgaben für Wunschpaare (hart, Min & Max)
PERCENT_PAIR_MIN = 60
PERCENT_PAIR_MAX = 85

# Optional: paar-spezifische Overrides
pair_percent_bounds = {
    # (mitarbeiter_index["Levi"], mitarbeiter_index["Lea"]): (70, 90),
}

# 🏖 Urlaub (harte Abwesenheiten)
urlaub = {
    5: [(date(2025, 10, 16), date(2025, 10, 31))],                                         # Lea
    6: [(date(2025, 10, 27), date(2025, 10, 31)), (date(2025, 12, 22), date(2026, 1, 5))],# Lena
    7: [(date(2025, 10, 16), date(2025, 10, 31))],                                         # Levi
}

def ist_ferien(datum: date) -> bool:
    return any(start <= datum <= ende for start, ende in ferien_bw)

def req_for(datum: date, tag: str, slot: str) -> int:
    profile = pflichtbesetzung_ferien if ist_ferien(datum) else pflichtbesetzung_normal
    return profile[tag][slot]

def ist_urlaub(e: int, datum: date) -> bool:
    return any(start <= datum <= ende for (start, ende) in urlaub.get(e, []))

def tage_index_liste(num_days, start_date, days_per_week):
    m = {tag: [] for tag in days_per_week}
    for d in range(num_days):
        tag = days_per_week[d % len(days_per_week)]
        m[tag].append(d)
    return m

success = False

for attempt in range(max_attempts):
    num_weeks = initial_weeks + attempt
    num_days  = num_weeks * len(days_per_week)

    # ----- Unverfügbarkeiten (hart) -----
    verboten = set()
    def tag_index(week, day):
        return week * len(days_per_week) + days_per_week.index(day)

    for week in range(num_weeks):
        # Alisa (0)
        for t in time_slots: verboten.add((0, tag_index(week,'Do'), t))
        for t in ['17','20']: verboten.add((0, tag_index(week,'Fr'), t))

        # Lena (6)
        for day in ['Mi','Do','Fr']:
            for t in time_slots: verboten.add((6, tag_index(week,day), t))
        verboten.add((6, tag_index(week,'Sa'), '20'))
        verboten.add((6, tag_index(week,'So'), '20'))

        # Maik (9)
        verboten.add((9, tag_index(week,'Do'), '20'))
        verboten.add((9, tag_index(week,'Fr'), '17'))
        verboten.add((9, tag_index(week,'Fr'), '20'))

        # Marianna (10)
        for t in ['14','17']:
            verboten.add((10, tag_index(week,'Mi'), t))
            verboten.add((10, tag_index(week,'Do'), t))
            verboten.add((10, tag_index(week,'Fr'), t))

        # Levi (7)
        for t in ['14','17']:
            verboten.add((7, tag_index(week,'Mi'), t))
            verboten.add((7, tag_index(week,'Do'), t))
            verboten.add((7, tag_index(week,'Fr'), t))
        verboten.add((7, tag_index(week,'Sa'), '14'))

        # Lea (5)
        for day in ['Mi','Do','Fr']:
            for t in ['14','17']: verboten.add((5, tag_index(week,day), t))
        verboten.add((5, tag_index(week,'Sa'), '14'))

        # Niko (13)
        for t in ['14','17']:
            verboten.add((13, tag_index(week,'Mi'), t))
            verboten.add((13, tag_index(week,'Fr'), t))
        for t in ['14','17','20']:
            verboten.add((13, tag_index(week,'Do'), t))
            verboten.add((13, tag_index(week,'Sa'), t))

        # Serkan (15)
        for t in ['14','17']:
            verboten.add((15, tag_index(week,'Mi'), t))
            verboten.add((15, tag_index(week,'Do'), t))
            verboten.add((15, tag_index(week,'Fr'), t))

        # Xenia (16)
        for t in ['14','17']:
            verboten.add((16, tag_index(week,'Mi'), t))
            verboten.add((16, tag_index(week,'Fr'), t))
            verboten.add((16, tag_index(week,'Sa'), t))
            verboten.add((16, tag_index(week,'So'), t))
        for t in time_slots:
            verboten.add((16, tag_index(week,'Do'), t))

        # Lucy (8)
        for t in time_slots:
            verboten.add((8, tag_index(week,'Mi'), t))
            verboten.add((8, tag_index(week,'Do'), t))

    # ----- Modell -----
    model = cp_model.CpModel()
    x = {}

    # ✅ Variablen nur, wenn Slot gebraucht; Urlaub/Unverfügbarkeit blocken
    for e in range(num_employees):
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                req = req_for(datum, tag, t)
                if req == 0:
                    continue
                var = model.NewBoolVar(f"x_{e}_{d}_{t}")
                x[(e, d, t)] = var
                if (e, d, t) in verboten:
                    model.Add(var == 0)
                if ist_urlaub(e, datum):
                    model.Add(var == 0)

    # Von hier IK Paare (harte Regel, nie zusammen in derselben Schicht)
    ik_paare = [
        ("Lea", "Wolfi"),
        ("Levi", "Wolfi"),
        ("Levi", "Kübra"),
        ("Levi", "Eren"),
        ("Lea", "Kübra"),
        ("Lea", "Eren"),
    ]
    # Namen prüfen & zu IDs mappen (mit Normalisierung)
    ik_pairs_idx = set()
    for a_name, b_name in ik_paare:
        if a_name not in mitarbeiter_index or b_name not in mitarbeiter_index:
            raise KeyError(f"ik_paare enthält unbekannte Namen: {(a_name, b_name)}")
        a = mitarbeiter_index[a_name]
        b = mitarbeiter_index[b_name]
        if a == b:
            raise ValueError(f"ik_paare enthält identische Person: {a_name}")
        if a > b: a, b = b, a
        ik_pairs_idx.add((a, b))

    # Constraints hinzufügen
    added_ik_constraints = 0
    for (i, j) in ik_pairs_idx:
        for d in range(num_days):
            for t in time_slots:
                if (i, d, t) in x and (j, d, t) in x:
                    model.Add(x[(i, d, t)] + x[(j, d, t)] <= 1)
                    added_ik_constraints += 1 #bis hier löschen

    # ===============================
    # ⏱️ Frequenz-Regeln (HARTE Regeln)
    # ===============================
    if frequency_rules:
        days_per_wk = len(days_per_week)

        # Hilfsfunktion: Liste der Ziel-(d,t) für eine Woche und eine Rule
        def targeted_slots_for_week(rule, w):
            d_start = w * days_per_wk
            d_end   = d_start + days_per_wk
            tgt_days  = set(rule.get("tags", days_per_week))
            tgt_times = set(rule["times"]) if rule.get("times") else set(time_slots)

            pairs = []
            for d in range(d_start, d_end):
                tag = days_per_week[d % days_per_wk]
                if tag not in tgt_days:
                    continue
                for t in tgt_times:
                    if (d < num_days) and ((t in time_slots) and req_for(date_for_d(d), tag, t) > 0) and any((e, d, t) in x for e in range(num_employees)):  # <— NEU
                        pairs.append((d, t))
            return pairs

        # Für jede Regel die Constraints bauen
        for rule in frequency_rules:
            window = int(rule.get("window_weeks", 4))
            cnt_mode = rule.get("count_per_week", "any")
            sliding  = bool(rule.get("sliding", False))

            if sliding:
                window_starts = range(0, max(0, num_weeks - window + 1))
            else:
                window_starts = range(0, num_weeks, window)

            week_targets = [targeted_slots_for_week(rule, w) for w in range(num_weeks)]

            for name in rule.get("employees", []):
                if name not in mitarbeiter_index:
                    raise KeyError(f"frequency_rules: unbekannte Person '{name}'")
                e = mitarbeiter_index[name]

                week_measures = []
                for w in range(num_weeks):
                    tgt_pairs = [(d, t) for (d, t) in week_targets[w] if (e, d, t) in x]

                    if cnt_mode == "any":
                        y = model.NewBoolVar(f"freq_any_{e}_w{w}")
                        if tgt_pairs:
                            for (d, t) in tgt_pairs:
                                model.Add(y >= x[(e, d, t)])
                            model.Add(y <= sum(x[(e, d, t)] for (d, t) in tgt_pairs))
                        else:
                            model.Add(y == 0)
                        week_measures.append((y, w))

                    elif cnt_mode == "slot":
                        s = model.NewIntVar(0, len(tgt_pairs), f"freq_slots_{e}_w{w}")
                        if tgt_pairs:
                            model.Add(s == sum(x[(e, d, t)] for (d, t) in tgt_pairs))
                        else:
                            model.Add(s == 0)
                        week_measures.append((s, w))
                    else:
                        raise ValueError(f"frequency_rules: unbekannter count_per_week='{cnt_mode}'")

                min_c = rule.get("min_count", None)
                max_c = rule.get("max_count", None)

                for w0 in window_starts:
                    w1 = min(w0 + window, num_weeks)
                    vars_in_window = [var for (var, ww) in week_measures if w0 <= ww < w1]
                    if not vars_in_window:
                        continue
                    if min_c is not None:
                        model.Add(sum(vars_in_window) >= int(min_c))
                    if max_c is not None:
                        model.Add(sum(vars_in_window) <= int(max_c))


    # 🤝 Paar-Variablen + Prozent-Constraint
    pair_vars = []
    pair_vars_by_pair = defaultdict(list)
    u_vars_by_pair    = defaultdict(list)

    if bevorzugte_paare_idx:
        for d in range(num_days):
            tag = days_per_week[d % len(days_per_week)]
            datum = date_for_d(d)  # <— NEU
            for t in time_slots:
                if req_for(datum, tag, t) < 2:
                    continue
                for (a, b) in bevorzugte_paare_idx:
                    va = x.get((a, d, t)); vb = x.get((b, d, t))
                    if va is None or vb is None:
                        continue
                    v = model.NewBoolVar(f"pair_{a}_{b}_d{d}_t{t}")
                    model.Add(v <= va); model.Add(v <= vb); model.Add(v >= va + vb - 1)
                    pair_vars.append(v); pair_vars_by_pair[(a, b)].append(v)

                    u = model.NewBoolVar(f"any_{a}_{b}_d{d}_t{t}")
                    model.Add(u >= va); model.Add(u >= vb); model.Add(u <= va + vb)
                    u_vars_by_pair[(a, b)].append(u)

    # 🔒 Min/Max-Prozentvorgaben für Wunschpaare (hart)
    for (aa, bb), v_list in pair_vars_by_pair.items():
        u_list = u_vars_by_pair[(aa, bb)]
        if not u_list:
            model.Add(sum(v_list) == 0)
            continue

        a, b = (aa, bb) if aa < bb else (bb, aa)
        S_v = sum(v_list)
        S_u = sum(u_list)

        min_p, max_p = pair_percent_bounds.get((a, b), (PERCENT_PAIR_MIN, PERCENT_PAIR_MAX))
        if (min_p is not None) and (max_p is not None) and (min_p > max_p):
            raise ValueError(f"Paar-Grenzen inkonsistent für {employees[a]} + {employees[b]}: {min_p}>{max_p}")

        if min_p is not None:
            model.Add(100 * S_v >= int(min_p) * S_u)
        if max_p is not None:
            model.Add(100 * S_v <= int(max_p) * S_u)

    # Wünsche sammeln (nur auf Pflichtslots)
    wunsch_vars = []
    for e in range(num_employees):
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                if req_for(datum, tag, t) == 0:
                    continue
                if e in wuensche and (tag, t) in wuensche[e]:
                    wunsch_vars.append(x[(e, d, t)])

    # ✅ Jede Pflichtschicht exakt decken (==)
    for d in range(num_days):
        datum = date_for_d(d)  # <— NEU
        tag = days_per_week[d % len(days_per_week)]
        for t in time_slots:
            req = req_for(datum, tag, t)
            if req == 0:
                continue
            bes = sum(x[(e, d, t)] for e in range(num_employees) if (e, d, t) in x)
            model.Add(bes == req)

    # 🔁 Tagesregeln
    for e in range(num_employees):
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag   = days_per_week[d % len(days_per_week)]
            ts_needed = [ts for ts in time_slots if req_for(datum, tag, ts) > 0]
            if ts_needed:
                model.Add(sum(x.get((e, d, ts), 0) for ts in ts_needed) <= 2)
                model.Add(x.get((e, d, '20'), 0) + x.get((e, d, '14'), 0) <= 1)
                model.Add(x.get((e, d, '20'), 0) + x.get((e, d, '17'), 0) <= 1)

    # 🚫 Keine Back-to-Back Einsätze: Mi→Do und Do→Fr verboten (indizes bleiben)
    for e in range(num_employees):
        for w in range(num_weeks):
            d_mi = w * len(days_per_week) + days_per_week.index('Mi')
            d_do = w * len(days_per_week) + days_per_week.index('Do')
            d_fr = w * len(days_per_week) + days_per_week.index('Fr')

            mi_slots = [x[(e, d_mi, t)] for t in time_slots if (e, d_mi, t) in x]
            do_slots = [x[(e, d_do, t)] for t in time_slots if (e, d_do, t) in x]
            if mi_slots and do_slots:
                model.Add(sum(mi_slots) + sum(do_slots) <= 1)

            do_slots2 = [x[(e, d_do, t)] for t in time_slots if (e, d_do, t) in x]
            fr_slots = [x[(e, d_fr, t)] for t in time_slots if (e, d_fr, t) in x]
            if do_slots2 and fr_slots:
                model.Add(sum(do_slots2) + sum(fr_slots) <= 1)

    # 💤 Erholungsregel
    days_per_wk = len(days_per_week)
    for e in range(num_employees):
        for w in range(num_weeks):
            d_fr = w*days_per_wk + days_per_week.index('Fr')
            d_sa = w*days_per_wk + days_per_week.index('Sa')
            d_so = w*days_per_wk + days_per_week.index('So')
            if (e, d_fr, '20') in x and (e, d_sa, '14') in x:
                model.Add(x[(e, d_fr, '20')] + x[(e, d_sa, '14')] <= 1)
            if (e, d_sa, '20') in x and (e, d_so, '14') in x:
                model.Add(x[(e, d_sa, '20')] + x[(e, d_so, '14')] <= 1)

    # Wochenende: 14 == 17 nur, wenn beide gebraucht
    for e in range(num_employees):
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag   = days_per_week[d % len(days_per_week)]
            if tag in ['Sa','So'] and req_for(datum, tag, '14') > 0 and req_for(datum, tag, '17') > 0:
                model.Add(x.get((e, d, '14'), 0) == x.get((e, d, '17'), 0))

    # 📊 Loads berechnen
    total_required_slots = 0
    for d in range(num_days):
        datum = date_for_d(d)  # <— NEU
        tag   = days_per_week[d % len(days_per_week)]
        for t in time_slots:
            total_required_slots += req_for(datum, tag, t)

    loads = []
    for e in range(num_employees):
        load_e = model.NewIntVar(0, total_required_slots, f"load_{e}")
        load_terms = []
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag   = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                if req_for(datum, tag, t) > 0 and (e, d, t) in x:
                    load_terms.append(x[(e, d, t)])
        model.Add(load_e == (sum(load_terms) if load_terms else 0))
        loads.append(load_e)

    # Spread (weich, per cap), Slottyp-Fairness (weich)
    max_load = model.NewIntVar(0, total_required_slots, "max_load")
    min_load = model.NewIntVar(0, total_required_slots, "min_load")
    for ld in loads:
        model.Add(ld <= max_load)
        model.Add(ld >= min_load)

    spread_caps = [2, 3, 3, 3]
    cap = spread_caps[min(attempt, len(spread_caps)-1)]
    model.Add(max_load - min_load <= cap)

    # Slottyp-Fairness: harte Caps + weiche Abweichung
    slot_cap_add_list = [1, 2, 3]
    add_cap = slot_cap_add_list[min(attempt, len(slot_cap_add_list)-1)]
    total_s_by_type = {(tag, t): 0 for tag in days_per_week for t in time_slots}
    for d in range(num_days):
        datum = date_for_d(d)  # <— NEU
        tag   = days_per_week[d % len(days_per_week)]
        for t in time_slots:
            total_s_by_type[(tag, t)] += req_for(datum, tag, t)

    slot_dev_vars = []
    tage_pro_tag = tage_index_liste(num_days, start_date, days_per_week)
    for tag in days_per_week:
        for t in time_slots:
            total_s = total_s_by_type[(tag, t)]
            if total_s == 0:
                continue
            q = total_s // num_employees
            hard_cap = q + add_cap
            for e in range(num_employees):
                y_terms = []
                for d in tage_pro_tag[tag]:
                    datum = date_for_d(d)  # <— NEU
                    if req_for(datum, tag, t) == 0:
                        continue
                    if (e, d, t) in x:
                        y_terms.append(x[(e, d, t)])
                if not y_terms:
                    continue
                y_expr = sum(y_terms)
                model.Add(y_expr <= hard_cap)

                z_hi = model.NewBoolVar(f"dev_hi_{e}_{tag}_{t}")
                z_lo = model.NewBoolVar(f"dev_lo_{e}_{tag}_{t}")
                model.Add(y_expr >= q + 2).OnlyEnforceIf(z_hi)
                model.Add(y_expr <= q + 1).OnlyEnforceIf(z_hi.Not())
                model.Add(y_expr <= q - 1).OnlyEnforceIf(z_lo)
                model.Add(y_expr >= q     ).OnlyEnforceIf(z_lo.Not())
                slot_dev_vars.extend([z_hi, z_lo])

    # ===============================
    # ⚖️ Fairness pro-rata (Urlaub) — harte Bandbreite
    # ===============================
    # 1) Verfügbare Pflichtslots je Person (Urlaub abgezogen)
    avail_required_slots = [0]*num_employees
    for e in range(num_employees):
        cnt = 0
        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            tag   = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                if req_for(datum, tag, t) == 0:
                    continue
                if not ist_urlaub(e, datum):
                    cnt += 1
        avail_required_slots[e] = cnt

    # 2) Pro-rata Zielwerte (gerundet) – basiert auf Verfügbarkeit ohne Urlaub
    sum_avail = sum(avail_required_slots)
    targets = [0]*num_employees
    for e in range(num_employees):
        if sum_avail > 0:
            targets[e] = int(round(total_required_slots * (avail_required_slots[e] / float(sum_avail))))
        else:
            targets[e] = 0

    # 3) Harte Bandbreite um das Ziel
    BAND = 1  # erlaubte Abweichung +/- BAND
    for e in range(num_employees):
        lower = max(0, targets[e] - BAND)
        upper = min(avail_required_slots[e], targets[e] + BAND)
        model.Add(loads[e] >= lower)
        model.Add(loads[e] <= upper)

    # ===============================
    # 🚫 Max 2 Schichten pro Woche
    # ===============================
    for e in range(num_employees):
        for w in range(num_weeks):
            week_slots = []
            for d in range(w * len(days_per_week), (w+1) * len(days_per_week)):
                datum = date_for_d(d)  # <— NEU
                tag   = days_per_week[d % len(days_per_week)]
                for t in time_slots:
                    if req_for(datum, tag, t) > 0 and (e, d, t) in x:
                        week_slots.append(x[(e, d, t)])
            if week_slots:
                model.Add(sum(week_slots) <= 2)

    # 🎯 Ziel
    objective_terms = []
    if wunsch_vars:
        objective_terms.append(W_WISH * sum(wunsch_vars))
    objective_terms.append(-W_SPREAD * (max_load - min_load))
    if slot_dev_vars:
        objective_terms.append(-W_SLOTDEV * sum(slot_dev_vars))
    if pair_vars:
        objective_terms.append(W_PAIR * sum(pair_vars))

    model.Maximize(sum(objective_terms))

    # 🚀 Solver: mehrere Seeds
    best_status = None
    for seed in range(5):
        solver = cp_model.CpSolver()
        solver.parameters.num_search_workers = 8
        solver.parameters.max_time_in_seconds = 240
        solver.parameters.random_seed = attempt * 100 + seed + 1
        status = solver.Solve(model)
        print(f"Attempt {attempt}.{seed+1}: {solver.StatusName(status)}, Conflicts={solver.NumConflicts()}")
        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            best_status = status
            break

    # 🆕 Attempt-Zeile mit Planwochen & ~Kalenderwochen
    end_date = date_for_d(num_days - 1)
    kal_wochen = ((end_date - start_date).days // 7) + 1
    print(f"📅 Attempt {attempt}: Planwochen={num_weeks} (Mi–So), Kalenderwochen≈{kal_wochen}, Spread≤{cap}")

    if best_status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        # ✅ Plan bauen
        plan = []
        for d in range(num_days):
            current_date = date_for_d(d)  # <— NEU
            tag = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                req = req_for(current_date, tag, t)
                if req == 0:
                    continue
                zugewiesen = [employees[e] for e in range(num_employees)
                              if (e, d, t) in x and solver.Value(x[(e, d, t)]) == 1]
                plan.append({
                    "Datum": current_date.strftime("%d.%m.%Y"),
                    "Tag": tag,
                    "Zeit": t,
                    "Mitarbeiter": ", ".join(zugewiesen)
                })

        df = pd.DataFrame(plan)
        print("\n📅 FINALER SCHICHTPLAN:")
        print(df.to_string(index=False))

        # ✅ Audit
        ok = True
        plan_slots = defaultdict(list)
        for row in plan:
            key = (row["Datum"], row["Tag"], row["Zeit"])
            names = [n.strip() for n in row["Mitarbeiter"].split(",") if n.strip()]
            plan_slots[key].extend(names)

        for d in range(num_days):
            datum = date_for_d(d)  # <— NEU
            datum_str = datum.strftime("%d.%m.%Y")
            tag = days_per_week[d % len(days_per_week)]
            for t in time_slots:
                req = req_for(datum, tag, t)
                if req == 0:
                    continue
                assigned = len(plan_slots.get((datum_str, tag, t), []))
                if assigned != req:
                    print(f"❌ Audit: {datum_str} {tag} {t} → {assigned}/{req}")
                    ok = False
        if ok:
            print("✅ Audit: Alle Pflichtschichten exakt besetzt.")

        # 📊 Verteilung total
        totals = Counter()
        for row in plan:
            for name in row["Mitarbeiter"].split(", "):
                if name.strip():
                    totals[name] += 1

        # --- Urlaubs-Clipping (NEU: horizon_end statt horizon_days) ---
        def clipped_days_in_periods(periods, horizon_start, horizon_end):
            if not periods:
                return set()
            days = set()
            for (s, e) in periods:
                s2 = max(s, horizon_start)
                e2 = min(e, horizon_end)
                if s2 <= e2:
                    k = s2
                    while k <= e2:
                        days.add(k)
                        k += timedelta(days=1)
            return days

        # Urlaubstage (nur für Anzeige) zusammenstellen
        vac_info = {}
        h_start = start_date
        h_end   = date_for_d(num_days - 1)  # <— NEU
        for e, name in enumerate(employees):
            periods = urlaub.get(e, [])
            vac_days = clipped_days_in_periods(periods, h_start, h_end)  # <— NEU
            vac_info[e] = {
                "days": len(vac_days),
                "periods": periods,
            }

        # Soll ohne Urlaub (alle gleich verteilt)
        ideal_without_vac = total_required_slots / len(employees)

        # 🎯 pro-rata Ziele (harte Bandbreite oben)
        print("\n📊 VERTEILUNG PRO MITARBEITENDEN (mit Urlaub + Fairness-Ziel):")
        for e, mit in sorted(enumerate(employees), key=lambda x: x[1]):
            v = vac_info.get(e, {"days": 0, "periods": []})
            fair_with_vac = targets[e]
            delta = fair_with_vac - ideal_without_vac
            if v["days"] > 0:
                # Nur die im Horizont liegenden Urlaubsanteile anzeigen (NEU)
                clip_ranges = []
                for (s, e_) in v["periods"]:
                    s2, e2 = max(s, h_start), min(e_, h_end)
                    if s2 <= e2:
                        clip_ranges.append(f"{s2:%d.%m.%Y}–{e2:%d.%m.%Y}")
                periods_str = ", ".join(clip_ranges) if clip_ranges else "–"

                print(
                    f"{mit}: {totals[mit]} Schichten  | Urlaub: {v['days']} Tage ({periods_str}) "
                    f"→ Fairness-Ziel: {fair_with_vac:.0f} (statt {ideal_without_vac:.1f}, Δ {delta:+.1f})"
                )
            else:
                print(
                    f"{mit}: {totals[mit]} Schichten  "
                    f"→ Fairness-Ziel: {fair_with_vac:.0f} (statt {ideal_without_vac:.1f}, Δ {delta:+.1f})"
                )

        # 🧩 VERTEILUNG je Slottyp
        print("\n🧩 VERTEILUNG je Slottyp (Tag/Zeit) pro Mitarbeiter – Summen:")
        per_type = defaultdict(lambda: Counter())
        for row in plan:
            tag, t = row["Tag"], row["Zeit"]
            for name in [n.strip() for n in row["Mitarbeiter"].split(",") if n.strip()]:
                per_type[(tag, t)][name] += 1
        for (tag, t) in [('Mi','20'),('Do','20'),('Fr','17'),('Fr','20'),
                         ('Sa','14'),('Sa','17'),('Sa','20'),
                         ('So','14'),('So','17'),('So','20')]:
            total_here = sum(
                req_for(date_for_d(d), days_per_week[d % len(days_per_week)], t)  # <— NEU
                for d in range(num_days)
                if days_per_week[d % len(days_per_week)] == tag
            )
            if total_here == 0:
                continue
            counts = per_type[(tag, t)]
            line = ", ".join(f"{mit}:{counts[mit]}" for mit in sorted(employees))
            print(f"{tag} {t}: {line}")

        # 🤝 Wunschpaar-Statistik
        print("\n🤝 Wunschpaare (Prozente & Denominator-Erklärung):")
        for (aa, bb) in bevorzugte_paare_idx:
            a, b = (aa, bb) if aa < bb else (bb, aa)

            v_list = pair_vars_by_pair.get((a, b), [])  # beide zusammen (nur in req≥2)
            u_list = u_vars_by_pair.get((a, b), [])     # mind. einer (nur in req≥2)

            together = sum(solver.Value(v) for v in v_list)          # |A∩B|
            any_eligible = sum(solver.Value(u) for u in u_list)      # |A∪B| über req≥2

            # Union über ALLE Slots (req≥1) direkt aus x-Variablen ermitteln
            union_all = 0
            for d in range(num_days):
                for t in time_slots:
                    va = x.get((a, d, t))
                    vb = x.get((b, d, t))
                    if va is None and vb is None:
                        continue
                    if (va is not None and solver.Value(va) == 1) or (vb is not None and solver.Value(vb) == 1):
                        union_all += 1

            # Individuelle Totals aus vorher berechnetem 'totals'
            A_name, B_name = employees[a], employees[b]
            A_tot, B_tot = totals[A_name], totals[B_name]

            # Sanity-Check: |A∪B| über alle Slots = |A| + |B| − |A∩B|
            calc_union_all = A_tot + B_tot - together

            # Paar-spezifische oder globale Grenzen (gelten für 'eligible2+'!):
            min_p, max_p = pair_percent_bounds.get((a, b), (PERCENT_PAIR_MIN, PERCENT_PAIR_MAX))
            min_req = math.ceil((min_p/100.0) * any_eligible) if (min_p is not None) else 0
            max_all_elig = math.floor((max_p/100.0) * any_eligible) if (max_p is not None) else any_eligible

            # Prozente
            pct_elig = (100.0 * together / any_eligible) if any_eligible > 0 else 0.0
            pct_all  = (100.0 * together / union_all)    if union_all    > 0 else 0.0

            status = "OK"
            if any_eligible > 0:
                if (min_p is not None) and (together < min_req):
                    status = "⚠ zu wenig gemeinsam (bezogen auf req≥2)"
                if (max_p is not None) and (together > max_all_elig):
                    status = "⚠ zu viel gemeinsam (bezogen auf req≥2)"

            # Hauptzeile (Constraint-Denominator = eligible2+)
            bounds_str = []
            if min_p is not None:
                bounds_str.append(f"min {min_p}%⇒≥{min_req}")
            if max_p is not None:
                bounds_str.append(f"max {max_p}%⇒≤{max_all_elig}")
            bounds_out = " | " + ", ".join(bounds_str) if bounds_str else ""

            print(f"{A_name} + {B_name}: together={together} / eligible2+={any_eligible}  ({pct_elig:.1f}%)"
                  f"{bounds_out}  → {status}")

            # Zusatzinfo zur Einordnung
            print(f"   ℹ totals: {A_name}={A_tot}, {B_name}={B_tot}  | allSlots (A∪B)={union_all} "
                  f"(= A+B−together → {A_tot}+{B_tot}−{together}={calc_union_all})  → Anteil an ALLEN Slots: {pct_all:.1f}%")

        success = True
        break

    print("⚠️ Nicht lösbar mit diesem Spread – lockere weiter.\n")

if not success:
    print("❌ Keine gültige Lösung nach allen Versuchen.")
    if DEBUG:
        print("\n🔍 DEBUG: Verfügbare Pflichtslots je Mitarbeitendem im letzten Attempt")
        erlaubte_map = Counter()
        for e in range(num_employees):
            erlaubte = 0
            for d in range(num_days):
                datum = date_for_d(d)  # <— NEU
                tag   = days_per_week[d % len(days_per_week)]
                for t in time_slots:
                    if req_for(datum, tag, t) > 0 and not ist_urlaub(e, datum) and (e, d, t) not in verboten:
                        erlaubte += 1
            erlaubte_map[employees[e]] = erlaubte
        for name in sorted(erlaubte_map):
            print(f"{name}: {erlaubte_map[name]} mögliche Pflichtslots")


Attempt 0.1: INFEASIBLE, Conflicts=1606
Attempt 0.2: INFEASIBLE, Conflicts=1606
Attempt 0.3: INFEASIBLE, Conflicts=1606
Attempt 0.4: INFEASIBLE, Conflicts=1606
Attempt 0.5: INFEASIBLE, Conflicts=1606
📅 Attempt 0: Planwochen=13 (Mi–So), Kalenderwochen≈13, Spread≤2
⚠️ Nicht lösbar mit diesem Spread – lockere weiter.

Attempt 1.1: INFEASIBLE, Conflicts=1
Attempt 1.2: INFEASIBLE, Conflicts=1
Attempt 1.3: INFEASIBLE, Conflicts=1
Attempt 1.4: INFEASIBLE, Conflicts=1
Attempt 1.5: INFEASIBLE, Conflicts=1
📅 Attempt 1: Planwochen=14 (Mi–So), Kalenderwochen≈14, Spread≤3
⚠️ Nicht lösbar mit diesem Spread – lockere weiter.



In [ ]:
!git clone https://github.com/LeviGPUnkt/kino-schichtplaner.git
%cd kino-schichtplaner

In [ ]:
!pip install ortools pandas


In [ ]:
%cd /content/kino-schichtplaner
